<h1>Smart Tour Guide Bot</h1>

**Project Title:**
Multimodal AI Travel Assistant

**Names:**

- Renad Naser
- ⁠Amjad Althagafi

**Objective:**

In a world with many transportations and countries, many people are becoming excited about the visiting new places and discover new things. 
Nowadays, the internet is full with many resources which make it difficult to find a specific information with being sure that it is correct or enough.

So we design and deploy an intelligent AI chatbot that answers user travel-related questions using voice or text, by extracting and summarizing data from global travel guide sources.

**Description:**

This project aims to build a multimodal RAG chatbot that allows users to ask questions and requests, like —“What can I visit in Paris in 2 days?”—and receive instant, accurate responses. The chatbot uses speech recognition, and summarize relevant information from tourist guides like Wikivoyage. It supports both text and voice input and provides responses using pre-trained transformer models.

**AI Techniques:**

	•	Speech Recognition (Whisper by OpenAI)
	•	Text Chunking & Embedding (LangChain + HuggingFace Transformers)
	•	Vector Search (ChromaDB or FAISS DB)
	•	Multimodal RAG system
	•	Conversational Agents with Memory (LangChain Agents)
	•	LangSmith for evaluation

**Applications:**

	•	Smart travel planning assistants
	•	Accessibility tools for visually/hearing-impaired travelers
	•	Educational tools for learning about geography and culture
	•	Chatbots for tourism websites or mobile travel apps

In [1]:
!pip install faiss-cpu

In [9]:
import os
from langchain_core.documents import Document
from typing import Dict, List, Any
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
from dotenv import load_dotenv
_ = load_dotenv()


In [4]:
from datasets import load_dataset

data = load_dataset("wikipedia", "20220301.simple", split='train[:100]', trust_remote_code=True)
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [5]:
data[4]

{'id': '9',
 'url': 'https://simple.wikipedia.org/wiki/Air',
 'title': 'Air',
 'text': "Air refers to the Earth's atmosphere. Air is a mixture of many gases and tiny dust particles. It is the clear gas in which living things live and breathe. It has an indefinite shape and volume. It has mass and weight, because it is matter. The weight of air creates atmospheric  pressure. There is no air in outer space. \n\nAir is a mixture of about 78% of nitrogen, 21% of oxygen, 0.9% of argon, 0.04% of carbon dioxide, and very small amounts of other gases.  There is an average of about 1% water vapour.\n\nAnimals live and need to breathe the oxygen in the air. In breathing, the lungs put oxygen into the blood, and send back carbon dioxide to the air. Plants need the carbon dioxide in the air to live. They give off the oxygen that we breathe. Without it we die of asphyxia.  \n\nWind is moving air, this is refreshing. This causes weather. \n\nAir can be polluted by some gases (such as carbon monoxide

In [6]:
def docs(article):
  return Document(
      page_content=article['text'],
      metadata={"title": article['title']}
  )

documents = []
for raw in data:
 documents.append(docs(raw))

documents
 

[Document(metadata={'title': 'April'}, page_content='April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril\'s flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nThe Month \n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other\'s last days are exactly 35 weeks (245 days) apart.\n\nIn common years, April starts on the same da

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(documents)
split_docs

[Document(metadata={'title': 'April'}, page_content="April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nThe Month"),
 Document(metadata={'title': 'April'}, page_content="The Month \n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (

In [11]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
)

In [12]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(split_docs, embed)

# 5. Create retriever
retriever = vectorstore.as_retriever()

In [13]:
vectorstore.save_local("faiss.index")

In [14]:
from langchain.llms import OpenAI
# from google.colab import userdata
# api_key = userdata.get('OPENAI_API')
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


C:\Users\riina\AppData\Local\Temp\ipykernel_16044\89507056.py:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


In [15]:
# retrieval method 1
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm ,  # Swap with HuggingFaceHub or others if needed
    retriever=retriever,
    return_source_documents=True
)

In [16]:
query = query = "What is paris?"
result = qa_chain({"query": query})

print("Answer:\n", result["result"])
print("\nSources:\n", result["source_documents"])

C:\Users\riina\AppData\Local\Temp\ipykernel_16044\3031500208.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Answer:
  Paris is the capital and largest city of France.

Sources:
 [Document(id='eef99cda-f28e-4291-bb50-4f7606dcffe9', metadata={'title': 'April'}, page_content='London, United Kingdom\n Madrid, Spain\n Paris, France\n Rotterdam, Netherlands\n Utrecht, Netherlands\n Zurich, Switzerland'), Document(id='8e7640dd-d73f-4e38-8c22-5061fbacaef2', metadata={'title': 'Algebra'}, page_content='History'), Document(id='1641b7e7-42a2-4dd8-ad76-101ff59094c6', metadata={'title': 'Afghanistan'}, page_content='History'), Document(id='5cba6846-5c8f-4bb9-8322-5c67c380dcb6', metadata={'title': 'Armenia'}, page_content='History')]


In [17]:
query = "can you make a trip plan for 2 days in paris?"
result = qa_chain({"query": query})

print("Answer:\n", result["result"])
print("\nSources:\n", result["source_documents"])


Answer:
  I'm sorry, I cannot make a trip plan as I am a text-based program. However, I can provide information about events and locations in Paris.

Sources:
 [Document(id='eef99cda-f28e-4291-bb50-4f7606dcffe9', metadata={'title': 'April'}, page_content='London, United Kingdom\n Madrid, Spain\n Paris, France\n Rotterdam, Netherlands\n Utrecht, Netherlands\n Zurich, Switzerland'), Document(id='72f5b569-2882-4c16-aacf-31f36fe7dde7', metadata={'title': 'December'}, page_content='Moveable and Non-Single Day Events'), Document(id='153bb91d-c159-42d9-8476-e5cd2112fa16', metadata={'title': 'August'}, page_content='Moveable and Monthlong events'), Document(id='5a27522c-4aee-48b9-a946-b28bfe0b0937', metadata={'title': 'Austria'}, page_content='Food')]
